# Photometry

In 2010, The TRAPPIST-South telescope (Gillon et al., 2011) was first installed and commissioned at the ESO La Silla Observatory, Chile. The TRAPPIST-South telescope is a robotic telescope with a 60 cm primary mirror with a f/8 Ritchey–Chrétien design attached to a German equatorial mount (all built and designed by the German company ASTELCO). Searching for transits on the 50th brightest southern very-late M-dwarfs (Gillon et al., 2013), this prototype survey led to the discovery of the TRAPPIST-1 system: 7 Earth-sized planets orbiting an M8 type star, 3 of which lying in its habitable zone.

In this notebook, we will reduce the raw images of the first ever detection of a transit on TRAPPIST-1 ([Gillon et al. 2015](https://arxiv.org/pdf/1605.07211)).

## Exploring the data folder

Let's start by exploring the folder of raw data. 

**1. Use the `FitsManager` object from prose to explore the folder**

In [25]:
# TODO

As you can see the folder contains several images, some of them used for calibration of the science frames. We can use the `FitsManager` to select images associated to a certain night together with the corresponding calibration files.

In [ ]:
files = fm.observation_files(...)

## Processing a reference image

In order to perform the photometric extraction of stars fluxes on all images, we will select a reference image from which we will extract:

- The stars positions, then reused and detected on all images (after being aligned to the reference)
- The global full-width at half-maximum (fwhm) of the PSF, to scale the apertures used in the aperture photometry block


In [ ]:
import numpy as np
from prose import FITSImage, Sequence, blocks

# reference is middle image
ref = FITSImage(...)

```{note}
[FITSImage](prose.FITSImage) is the primary function used to load a FITS image into an [Image](prose.Image) object. While other loaders can be used, this loader is compatible with most of the images aquired from small to large-size ground-based telescopes.
```

**2. Look into the [photometry tutorial](https://prose.readthedocs.io/en/latest/ipynb/photometry.html) of prose to create and run the calibration [Sequence](https://prose.readthedocs.io/en/latest/md/generated/prose.Sequence.html#prose.Sequence) on the reference image**

## Identifying the target

In this image, we may want to identify the target by making a query to a catalog such as Gaia. During the original TRAPPIST, TRAPPIST-1 (located in this image) had not be named yet. One way to identify the target in the field was to check its coordinates and plot it on top of the image. Let's do that here.

**3.**
- **Find the coordinates of TRAPPIST-1** 
- **Create a [SkyCoord](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) object**
- **Transform the coordinates to pixel coordinates**
- **Plot the image and the target coordinates**

In [26]:
# TODO

target_id = ...

## Aperture photometry

We can now extract the photometry of these stars using this sequence

**4. Look into the [photometry tutorial](https://prose.readthedocs.io/en/latest/ipynb/photometry.html) to create and execute the reduction sequence on all images. Call this sequence `photometry`**

In [27]:
# TODO

In this sequence, we performed aperture photometry with a bunch of other image processing steps. Let plot the different apertures we used.

In [ ]:
# we isolate the aperture photometry block
ap_phot_block: blocks.AperturePhotometry = ...
annulus_block: blocks.AnnulusBackground = ...
radii = ap_phot_block._radii.copy()

ref_cutout = ref.cutout(target_id, 80)
ref_cutout.show(sources=False, figsize=5)
for r in radii * ref.fwhm:
    ref_cutout.sources[0].plot(r, label=False, c="C0")

ref_cutout.sources[0].plot(annulus_block.rin * ref.fwhm, label=False, c="y")
ref_cutout.sources[0].plot(annulus_block.rout * ref.fwhm, label=False, c="y")

- In yellow: we plotted the size of the annulus used to compute the local background of the target.
- In blue: we plotted all the apertures on which we extract the photometry.

```{note}
As you may notice, the apertures do not seem optimally set for our observation. This is expected as the default values of the `AperturePhotometry` and `AnnulusBackground` are set to accommodate very generic observations. As an exercise, try to specify some optimal apertures and annulus for this observation when instantiating these two blocks in the sequence.
```

## The [Fluxes](prose.Fluxes) object

The [GetFluxes](prose.blocks.GetFluxes) block provide a way to retrieve the fluxes computed by the [AperturePhotometry](prose.blocks.AperturePhotometry) minus the background estimated with [AnnulusBackground](prose.blocks.AnnulusBackground)

In [ ]:
from prose import Fluxes

raw_fluxes: Fluxes = photometry[-1].fluxes
fluxes = raw_fluxes.copy()

**5. Look into the [photometry tutorial](https://prose.readthedocs.io/en/latest/ipynb/photometry.html) to retrieve the fluxes from the science images and produce a differential light curve of the target**

In [ ]:
# TODO

Let's save these data to analyze them in the next notebook

In [ ]:
diff.save(f"../data/{ref.label}.flux")

## Finding the proper citations

`prose` depends on a lot of Python packages and published methods. When using prose in your research, you can generate a complete aknowledgment sentence (and associated BibTeX entries) based on the blocks used in a [Sequence](prose.Sequence).

For example, given our photometry sequence, citations can be extracted with


In [ ]:
acknowledgement, bibtex = photometry.citations()

where `acknowledgement` contains the aknowledgment sentence and `bibtex` the necessary BibTeX entries. Please make sure to check the generated text and BibTeX entries before using them in your work.

In [ ]:
print(acknowledgement, "\n")
print(bibtex[:500], "...")